In [55]:
import pandas as pd
import unicodedata

In [56]:
def remover_espacos_acentos(texto):
    if isinstance(texto, str):
        texto = texto.strip().replace(' ', '_')
        texto = ''.join(caracter for caracter in unicodedata.normalize('NFKD', texto) if not unicodedata.combining(caracter))
    return texto

def renomear_coluna(df):
    df = df.copy()
    ultimos_valores = df.iloc[-1]
    for coluna, valor in ultimos_valores.items():
        if coluna != 'ESTUDANTE':
            novo_nome = f"QT_{coluna}_{remover_espacos_acentos(valor)}"
            df = df.rename(columns={coluna: novo_nome})
    
    return df
def gerar_pontuacao_simulado(df):
    df = df.copy()
    colunas_inteiras = df.select_dtypes(include='int32').columns
    df['QTD_ACERTOS'] = df[colunas_inteiras].sum(axis=1)
    return df
def calcular_acertos_temas(df):
    df = df.copy()
    terminacao_subgrupo = {
        'QUIMICA': 'ACERTOS_CN',
        'FISICA': 'ACERTOS_CN',
        'BIOLOGIA': 'ACERTOS_CN',
        'HISTORIA':'ACERTOS_CH',
        'GEOGRAFIA':'ACERTOS_CH',
        'FILOSOFIA':'ACERTOS_CH',
        'SOCIOLOGIA':'ACERTOS_CH',
        'LITERATURA': 'ACERTOS_LG',
        'INTERTEXTO': 'ACERTOS_LG',
        'PORTUGUES': 'ACERTOS_LG',
        'INGLES':'ACERTOS_LG',
        'MATEMATICA': 'ACERTOS_MT'
    }
    subgrupos_somas = {}
    for coluna in df.columns:
        for terminacao, subgrupo in terminacao_subgrupo.items():
            if coluna.endswith(terminacao):
                if subgrupo not in subgrupos_somas:
                    subgrupos_somas[subgrupo] = df[coluna]
                else:
                    subgrupos_somas[subgrupo] += df[coluna]
                break
    df_somas = pd.DataFrame(subgrupos_somas)
    
    return df_somas
def remover_espacos_colunas(df):
    for coluna in df.columns:
        if isinstance(coluna, str):  
            novo_nome = coluna.strip().replace(' ', '')  
            df.rename(columns={coluna: novo_nome}, inplace=True)  
    return df
def extrair_abas_excel(nome_arquivo):
    xls = pd.ExcelFile(nome_arquivo)
    abas_dict = {}
    for aba_nome in xls.sheet_names:
        df = xls.parse(aba_nome)
        aba_nome = aba_nome.strip()
        abas_dict[aba_nome] = remover_espacos_colunas(df)
    return abas_dict
def verificar_respostas(df):
    df = df.copy()
    gabarito = df.iloc[-1]
    for coluna in df.columns:
        if coluna != "ESTUDANTE":
            df[coluna] = (df[coluna] == gabarito[coluna]).astype(int)
    return df.dropna()

    

# Correção dos simulados

In [57]:
gabaritos_simulado = extrair_abas_excel('data/raw/Gabaritos_AMPs_2023.xlsx')

## Simulado AMP 1.2

In [58]:
simulado_1 = pd.read_excel('data/raw/Respostas_AMP_1.2_2023.xlsx')

In [59]:
simulado_1_completo = simulado_1.merge(gabaritos_simulado['AMP_1.2'],how='outer')

In [60]:
simulado_1_parcial = renomear_coluna(simulado_1_completo)

In [61]:
simulado_1_parcial = verificar_respostas(simulado_1_parcial.loc[:simulado_1_parcial.shape[0]-2])


In [62]:
simulado_1_final = pd.concat([gerar_pontuacao_simulado(simulado_1_parcial), calcular_acertos_temas(simulado_1_parcial)],axis=1)

## Simulado AMP 2.1

In [63]:
simulado_2 = pd.read_excel('data/raw/Respostas_AMP_2.1_2023.xlsx')

In [64]:
simulado_2.drop('Tipo de Prova',axis=1,inplace=True)

In [65]:
simulado_2_completo = simulado_2.merge(gabaritos_simulado['AMP_2.1'],how='outer')

In [66]:
simulado_2_parcial = renomear_coluna(simulado_2_completo)

In [67]:
simulado_2_parcial = verificar_respostas(simulado_2_parcial.loc[:simulado_2_parcial.shape[0]-2])


In [68]:
simulado_2_final = pd.concat([gerar_pontuacao_simulado(simulado_2_parcial), calcular_acertos_temas(simulado_2_parcial)],axis=1)

In [69]:
simulado_2_final

,ESTUDANTE,QT_1_HISTORIA,QT_2_GEOGRAFIA,QT_3_FILOSOFIA,QT_4_HISTORIA,QT_5_GEOGRAFIA,QT_6_FILOSOFIA,QT_7_HISTORIA,QT_8_GEOGRAFIA,QT_9_FILOSOFIA,...,QT_84_PORTUGUES,QT_85_PORTUGUES,QT_86_PORTUGUES,QT_87_PORTUGUES,QT_88_PORTUGUES,QT_89_PORTUGUES,QT_90_PORTUGUES,QTD_ACERTOS,ACERTOS_CH,ACERTOS_LG
0,Maria Luiza da Silva,0,0,0,0,0,1,1,1,0,...,1,1,0,0,1,1,1,44,22,22
1,João Pedro Santos,0,0,1,1,1,0,0,1,0,...,1,1,0,0,0,0,1,38,13,25
2,Ana Clara Oliveira,0,0,0,0,1,1,0,0,1,...,0,1,0,0,0,0,1,21,9,12
3,Lucas Gabriel Costa,0,0,0,0,1,0,0,1,1,...,0,0,1,0,1,1,0,22,12,10
4,Juliana Cristina Almeida,0,0,0,1,0,1,0,1,1,...,1,1,0,0,1,0,1,47,22,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Juliana Alves Costa,0,0,1,0,1,0,0,1,0,...,0,1,0,0,0,0,0,30,15,15
81,Paulo Nunes Ribeiro,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,1,0,19,9,10
82,Renata Oliveira Mendonça,0,0,0,0,0,0,0,1,1,...,0,1,0,0,0,0,1,32,15,17
83,Gustavo Silva Carvalho,0,0,0,0,0,0,0,0,0,...,1,1,0,0,1,0,0,30,12,18


## Simulado AMP 2.2

In [70]:
simulado_3 = pd.read_excel('data/raw/Respostas_AMP_2.2_2023.xlsx')

In [71]:
simulado_3.drop('Tipo de Prova',axis=1,inplace=True)

In [72]:
simulado_3_completo = simulado_3.merge(gabaritos_simulado['AMP_2.2'],how='outer')

In [73]:
simulado_3_parcial = renomear_coluna(simulado_3_completo)

In [74]:
simulado_3_parcial = verificar_respostas(simulado_3_parcial.loc[:simulado_3_parcial.shape[0]-2])


In [75]:
simulado_3_final = pd.concat([gerar_pontuacao_simulado(simulado_3_parcial), calcular_acertos_temas(simulado_3_parcial)],axis=1)

## Exportando 

In [76]:
simulado_1_final.to_csv('data/serving/simulado_final_1.2.csv',index=False)

In [77]:
simulado_2_final.to_csv('data/serving/simulado_final_2.1.csv',index=False)

In [78]:
simulado_3_final.to_csv('data/serving/simulado_final_2.2.csv',index=False)